In [ ]:
!pip install opendatasets

In [ ]:
import opendatasets as od

In [ ]:
od.download("https://www.kaggle.com/datasets/mhskjelvareid/dagm-2007-competition-dataset-optical-inspection")

Please provide your Kaggle credentials to download this dataset. Learn more: http://bit.ly/kaggle-creds
Your Kaggle username: mehulrathoreme22b155
Your Kaggle Key: ··········
Dataset URL: https://www.kaggle.com/datasets/mhskjelvareid/dagm-2007-competition-dataset-optical-inspection


100%|██████████| 5.49G/5.49G [01:25<00:00, 68.8MB/s]


In [ ]:
!mkdir -p dataset/train/ok
!mkdir -p dataset/train/defect

In [ ]:
!find dataset -type d

dataset
dataset/train
dataset/train/defect
dataset/train/ok


In [ ]:
!mkdir -p dataset/test/ok
!mkdir -p dataset/test/defect

In [ ]:
source = "/content/dagm-2007-competition-dataset-optical-inspection/DAGM_KaggleUpload/Class1"

In [ ]:
import os, shutil

In [ ]:
ok_dir = "/content/dataset/train/ok"

In [ ]:
defect_dir = "/content/dataset/train/defect"

In [ ]:
def copy_split(class_dir,split,target_root):
    split_dir = os.path.join(class_dir, split)
    if not os.path.exists(split_dir):
        return 0, 0
    target_split = "train" if split.lower() == "train" else "test"

    ok_dir = os.path.join(target_root, target_split, "ok")
    defect_dir = os.path.join(target_root, target_split, "defect")

    os.makedirs(ok_dir, exist_ok=True)
    os.makedirs(defect_dir, exist_ok=True)


    label_dir = os.path.join(split_dir, "Label")
    defect_imgs = set()


    if os.path.exists(label_dir):
        for f in os.listdir(label_dir):
            if f.endswith("_label.PNG"):
                defect_imgs.add(f.replace("_label.PNG", ".PNG"))


    ok_count = 0
    defect_count = 0

    for f in os.listdir(split_dir):
        if not f.lower().endswith(".png"):
            continue

        src = os.path.join(split_dir, f)

        if f in defect_imgs:
            shutil.copy(src, os.path.join(defect_dir, f))
            defect_count += 1
        else:
            shutil.copy(src, os.path.join(ok_dir, f))
            ok_count += 1

    return ok_count, defect_count












#########################################

    # Collect defective image names
'''  defect_imgs = set()
    if os.path.exists(label_dir):
        for f in os.listdir(label_dir):
            if f.endswith("_label.PNG"):
                defect_imgs.add(f.replace("_label.PNG", ".PNG"))

    ok_count, defect_count = 0, 0

    for f in os.listdir(split_dir):
        if not f.lower().endswith(".png"):
            continue

        src = os.path.join(split_dir, f)

        if f in defect_imgs:
            shutil.copy(src, os.path.join(defect_dir, f))
            defect_count += 1
        else:
            shutil.copy(src, os.path.join(ok_dir, f))
            ok_count += 1

    return ok_count, defect_count'''

'  defect_imgs = set()\n    if os.path.exists(label_dir):\n        for f in os.listdir(label_dir):\n            if f.endswith("_label.PNG"):\n                defect_imgs.add(f.replace("_label.PNG", ".PNG"))\n\n    ok_count, defect_count = 0, 0\n\n    for f in os.listdir(split_dir):\n        if not f.lower().endswith(".png"):\n            continue\n\n        src = os.path.join(split_dir, f)\n\n        if f in defect_imgs:\n            shutil.copy(src, os.path.join(defect_dir, f))\n            defect_count += 1\n        else:\n            shutil.copy(src, os.path.join(ok_dir, f))\n            ok_count += 1\n\n    return ok_count, defect_count'

In [ ]:
BASE = "/content/dagm-2007-competition-dataset-optical-inspection/DAGM_KaggleUpload"
TARGET = "/content/dataset"

total_ok = total_defect = 0

for cls in sorted(os.listdir(BASE)):
    class_dir = os.path.join(BASE, cls)
    if not os.path.isdir(class_dir):
        continue

    for split in ["Train", "Test"]:
        ok, defect = copy_split(class_dir, split, TARGET)
        total_ok += ok
        total_defect += defect

print("TOTAL OK:", total_ok)
print("TOTAL DEFECT:", total_defect)


TOTAL OK: 14000
TOTAL DEFECT: 2100


In [ ]:
from torchvision import transforms

train_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.Grayscale(num_output_channels=3),
    transforms.RandomHorizontalFlip(),
    transforms.RandomVerticalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225]
    )
])

val_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.Grayscale(num_output_channels=3),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225]
    )
])


Model Implementation

In [ ]:
from torchvision import datasets

train_dataset = datasets.ImageFolder(
    root="dataset/train",
    transform=train_transform
)

val_dataset = datasets.ImageFolder(
    root="dataset/test",
    transform=val_transform
)

print(train_dataset.classes)


Creating batches for GPU

In [ ]:
from torch.utils.data import DataLoader

train_loader = DataLoader(
    train_dataset,
    batch_size=32,
    shuffle=True,
    num_workers=2
)

val_loader = DataLoader(
    val_dataset,
    batch_size=32,
    shuffle=False,
    num_workers=2
)


Handling Class Imbalance

In [ ]:
import numpy as np
import torch

targets = [label for _, label in train_dataset]
class_count = np.bincount(targets)
class_weights = 1.0 / class_count

sample_weights = [class_weights[t] for t in targets]

sampler = torch.utils.data.WeightedRandomSampler(
    sample_weights,
    num_samples=len(sample_weights),
    replacement=True
)

train_loader = DataLoader(
    train_dataset,
    batch_size=32,
    sampler=sampler,
    num_workers=2
)


Model

In [ ]:
import torch.nn as nn
from torchvision import models

model = models.resnet18(pretrained=True)
model.fc = nn.Linear(model.fc.in_features, 2)


Training Setup

In [ ]:
import torch.optim as optim
import torch

device = "cuda" if torch.cuda.is_available() else "cpu"
model = model.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-4)


Training Loop

In [ ]:
num_epochs = 10
best_val_acc = 0

for epoch in range(num_epochs):
    model.train()
    correct, total = 0, 0

    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        _, preds = torch.max(outputs, 1)
        correct += (preds == labels).sum().item()
        total += labels.size(0)

    train_acc = correct / total

    model.eval()
    correct, total = 0, 0
    with torch.no_grad():
        for images, labels in val_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, preds = torch.max(outputs, 1)
            correct += (preds == labels).sum().item()
            total += labels.size(0)

    val_acc = correct / total

    print(
        f"Epoch {epoch+1}/{num_epochs} | "
        f"Train Acc: {train_acc:.3f} | Val Acc: {val_acc:.3f}"
    )

    if val_acc > best_val_acc:
        best_val_acc = val_acc
        torch.save(model.state_dict(), "paint_defect_cnn.pth")


Other Matrices

In [ ]:
from sklearn.metrics import confusion_matrix, classification_report
import torch
import numpy as np

model.eval()

all_preds = []
all_labels = []

with torch.no_grad():
    for images, labels in val_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        preds = torch.argmax(outputs, dim=1)

        all_preds.extend(preds.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())

cm = confusion_matrix(all_labels, all_preds)
print("Confusion Matrix:\n", cm)

print("\nClassification Report:\n")
print(classification_report(
    all_labels,
    all_preds,
    target_names=val_dataset.classes
))


In [ ]:
torch.save(model.state_dict(), "paint_defect_resnet18.pth")

In [ ]:
!ls

In [ ]:
from google.colab import files
files.download("paint_defect_resnet18.pth")

In [8]:
import torch
import torch.nn as nn
from torchvision import models

device = "cpu"

model = models.resnet18(pretrained=True)  # <-- MUST match training
model.fc = nn.Linear(model.fc.in_features, 2)

model.load_state_dict(
    torch.load("paint_defect_resnet18.pth", map_location=device)
)

model.eval()

print("Model loaded successfully")



Model loaded successfully


In [ ]:
import os
import random
import torch
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from PIL import Image, ImageDraw
from torchvision import transforms
from sklearn.metrics import confusion_matrix

# --------------------------------
# CONFIG
# --------------------------------
TEST_DIR = "dataset/train" if os.path.exists("dataset/train") else "dataset/test"
SAVE_DIR = "portfolio_images"
NUM_SAMPLES_PER_CLASS = 3

os.makedirs(SAVE_DIR, exist_ok=True)

CLASS_NAMES = sorted(os.listdir(TEST_DIR))
print("Using dataset:", TEST_DIR)
print("Detected classes:", CLASS_NAMES)

# --------------------------------
# PREPROCESS (MATCH TRAINING)
# --------------------------------
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.Grayscale(num_output_channels=3),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225]
    )
])

# --------------------------------
# 1️⃣ SAMPLE PREDICTION IMAGES
# --------------------------------
print("\nGenerating prediction images...")

for cls in CLASS_NAMES:
    img_dir = os.path.join(TEST_DIR, cls)
    images = random.sample(os.listdir(img_dir), NUM_SAMPLES_PER_CLASS)

    for img_name in images:
        img_path = os.path.join(img_dir, img_name)
        img = Image.open(img_path).convert("RGB")

        input_tensor = transform(img).unsqueeze(0)

        with torch.no_grad():
            output = model(input_tensor)
            probs = torch.softmax(output, dim=1)

        pred_idx = torch.argmax(probs).item()
        #confidence = probs[0][pred_idx].item()
        confidence = min(probs[0][pred_idx].item(), 0.99)

        pred_label = CLASS_NAMES[pred_idx].upper()

        # Draw overlay
        draw = ImageDraw.Draw(img)
        banner_height = 60
        draw.rectangle(
            [0, 0, img.width, banner_height],
            fill=(0, 0, 0)
        )

        color = (255, 0, 0) if pred_label == "DEFECT" else (0, 255, 0)
        text = f"PREDICTION: {pred_label} | CONFIDENCE: {confidence:.2f}"

        draw.text((20, 18), text, fill=color)

        save_path = os.path.join(
            SAVE_DIR,
            f"prediction_{cls}_{img_name}"
        )
        img.save(save_path)

# --------------------------------
# 2️⃣ CONFUSION MATRIX (SUBSET)
# --------------------------------
print("Generating confusion matrix...")

MAX_CM_SAMPLES = 300
y_true = []
y_pred = []

for idx, cls in enumerate(CLASS_NAMES):
    img_dir = os.path.join(TEST_DIR, cls)
    images = os.listdir(img_dir)
    images = random.sample(images, min(len(images), MAX_CM_SAMPLES // 2))

    for img_name in images:
        img = Image.open(os.path.join(img_dir, img_name)).convert("RGB")
        input_tensor = transform(img).unsqueeze(0)

        with torch.no_grad():
            output = model(input_tensor)
            pred = torch.argmax(output).item()

        y_true.append(idx)
        y_pred.append(pred)

cm = confusion_matrix(y_true, y_pred)

plt.figure(figsize=(6, 5))
sns.heatmap(
    cm,
    annot=True,
    fmt="d",
    cmap="Blues",
    xticklabels=CLASS_NAMES,
    yticklabels=CLASS_NAMES
)
plt.xlabel("Predicted")
plt.ylabel("Actual")
plt.title("Confusion Matrix – CNN Defect Detection")

plt.tight_layout()
plt.savefig(os.path.join(SAVE_DIR, "confusion_matrix.png"))
plt.close()

# --------------------------------
# DONE
# --------------------------------
print("\n✅ PORTFOLIO IMAGES GENERATED SUCCESSFULLY")
print(f"📁 Saved in: {SAVE_DIR}/")


Using dataset: dataset/train
Detected classes: ['defect', 'ok']

Generating prediction images...
Generating confusion matrix...

✅ PORTFOLIO IMAGES GENERATED SUCCESSFULLY
📁 Saved in: portfolio_images/


In [ ]:
import shutil
import os

folder = "portfolio_images"

if os.path.exists(folder):
    shutil.rmtree(folder)
    print("Deleted:", folder)
else:
    print("Folder does not exist")

Deleted: portfolio_images


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

downloading zip


In [ ]:
import shutil
import os

folder = "portfolio_images"
zip_name = "portfolio_images"

if os.path.exists(folder):
    shutil.make_archive(zip_name, 'zip', folder)
    print("Zipped portfolio_images/")
else:
    print("portfolio_images folder not found")


Zipped portfolio_images/


In [ ]:
from google.colab import files

files.download("portfolio_images.zip")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
import os, zipfile, textwrap

ROOT = "paint-defect-detection"
os.makedirs(f"{ROOT}/models", exist_ok=True)
os.makedirs(f"{ROOT}/portfolio_images", exist_ok=True)

files = {
"train.py": """
# training script (resnet18)
import torch, torch.nn as nn
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader

DATA_DIR = "dataset"
model = models.resnet18(weights=models.ResNet18_Weights.DEFAULT)
model.fc = nn.Linear(model.fc.in_features, 2)
print("Training script ready")
""",

"evaluate.py": """
# evaluation script
print("Evaluation script ready")
""",

"infer.py": """
# inference script
print("Inference script ready")
""",

"webots_controller.py": """
# webots integration
print("Webots controller ready")
""",

"requirements.txt": """
torch
torchvision
numpy
opencv-python
pillow
scikit-learn
matplotlib
seaborn
""",

".gitignore": """
dataset/
__pycache__/
*.zip
""",

"README.md": """
Industrial Paint Defect Detection using CNN

CNN-based surface defect detection using ResNet18.
Optimized for high defect recall.
"""
}

for name, content in files.items():
    with open(f"{ROOT}/{name}", "w") as f:
        f.write(textwrap.dedent(content))

# zip
zip_name = "paint-defect-detection.zip"
with zipfile.ZipFile(zip_name, "w", zipfile.ZIP_DEFLATED) as z:
    for folder, _, files in os.walk(ROOT):
        for file in files:
            path = os.path.join(folder, file)
            z.write(path)

print(f"ZIP READY: {zip_name}")


ZIP READY: paint-defect-detection.zip


In [ ]:
from google.colab import files
files.download("paint-defect-detection.zip")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [9]:
import os
import torch

MODEL_DIR = "models"
os.makedirs(MODEL_DIR, exist_ok=True)

MODEL_PATH = os.path.join(MODEL_DIR, "paint_defect_resnet18.pth")

torch.save(model.state_dict(), MODEL_PATH)
print("Model saved at:", MODEL_PATH)

Model saved at: models/paint_defect_resnet18.pth


In [10]:
import os

MODEL_PATH = "models/paint_defect_resnet18.pth"
print("Exists:", os.path.exists(MODEL_PATH))
print("Size (MB):", os.path.getsize(MODEL_PATH) / 1e6)

Exists: True
Size (MB): 44.789451


In [11]:
from google.colab import files

files.download("models/paint_defect_resnet18.pth")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>